In [1]:
import os
import numpy as np
import librosa
import pickle
from scipy.fftpack import dct

In [ ]:
# Xác định các paths
DATASET_PATH = "D:/VOICE_CONVERTED/DemoProject/Run/RunFlac"
LABEL_FILE_PATH = "D:/VOICE_CONVERTED/DemoProject/Run/file_label_run.txt"
# Xác định các thông số
NUM_CLASSES = 2  # số nhãn
SAMPLE_RATE = 16000  # SR của file âm thanh
DURATION = 5  # Thời gian file âm thanh
N_MELS = 128  # Số Mel Bin
N_COF = 29

In [ ]:
def load_audio(filepath, sr=None):
    y, sr = librosa.load(filepath, sr=sr)
    return y, sr

# tính toán constant-Q
def compute_cqt(y, sr, n_bins=84, bins_per_octave=12):
    cqt = np.abs(librosa.cqt(y, sr=sr, n_bins=n_bins, bins_per_octave=bins_per_octave)) ** 2
    return cqt

# Logarith hóa
def log_compress(spectrum):
    return np.log1p(spectrum)


# chuẩn hóa chieuf dữ liêu
def uniform_sampling(log_spectrum, num_samples=50):
    indices = np.linspace(0, log_spectrum.shape[0] - 1, num_samples, dtype=int)
    return log_spectrum[indices, :]

# tính dct
def compute_dct(log_cqt):
    cqcc = dct(log_cqt, type=2, axis=0, norm='ortho')
    return cqcc

def extract_cqcc(y, sr, num_samples=60, num_coefficients=29):
    cqt = compute_cqt(y, sr)
    log_cqt = log_compress(cqt)
    sampled_log_cqt = uniform_sampling(log_cqt, num_samples)
    
    # DCT 
    cqcc = compute_dct(sampled_log_cqt)
    
    # Lấy những bin đầu của tần âm phổ,...
    return cqcc[:num_coefficients, :]

In [4]:
# Lấy Labels của files
labels = {}
with open(LABEL_FILE_PATH, 'r') as label_file:
    lines = label_file.readlines()

for line in lines:
    parts = line.strip().split()
    file_name = parts[1]
    label = 1 if parts[-1] == "bonafide" else 0
    labels[file_name] = label


In [6]:
X = []
y = []

# Lấy features Mel-spectrogram của files âm thanh
max_time_steps = 109  # max timestep của model
for file_name, label in labels.items():
    file_path = os.path.join(DATASET_PATH, file_name + ".flac")

    # Tải âm thành
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)

    cqcc = extract_cqcc(audio, SAMPLE_RATE, N_COF)

    if cqcc.shape[1] < max_time_steps:
        cqcc = np.pad(cqcc, ((0, 0), (0, max_time_steps - cqcc.shape[1])), mode='constant')
    else:
        cqcc = cqcc[:, :max_time_steps]

    X.append(cqcc)
    y.append(label)

In [7]:
# Chueyern dữ liệu list về numpy
X = np.array(X)
y = np.array(y)

In [ ]:
# Lưu Label và Features của files âm thanh
np.save('./SavedFeatures/X_cqcc4.npy', X)

In [9]:
np.save('./SavedFeatures/y_cqcc4.npy', y)